# Extracting Features with ViT
- https://huggingface.co/docs/transformers/model_doc/vit#vision-transformer-vit
- https://arxiv.org/abs/2010.11929

In [1]:
import os
import torch
# import csv

import pandas as pd
import numpy as np
# import torchvision.transforms as transforms

# from PIL import Image
# from transformers import ViTImageProcessor, ViTFeatureExtractor, ViTModel
# from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
BASE = '/Users/brinkley97/Documents/development/'
CLASS_PATH = 'classes/csci_535_multimodal_probabilistic_learning/'
DATASET_PATH = 'datasets/'

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

In [ ]:
def load_data(file):
    original_data = pd.read_csv(file)
    # original_data = pd.DataFrame(file)
    copy_of_data = original_data.copy()
    return copy_of_data

In [ ]:
file_paths = BASE + CLASS_PATH + DATASET_PATH + 'cremaD_mspR_mspT.csv'

In [ ]:
dataset_paths_copy = load_data(file_paths)
# dataset_paths_copy

# MSP

In [ ]:
MSP_DATASET_PATH = 'msp/videos/r_and_t_frames/'

## T

In [ ]:
msp_t_paths = dataset_paths_copy['MSP T Paths']
type(msp_t_paths), msp_t_paths

In [ ]:
sub_set_msp_t_paths = msp_t_paths.loc[0:300]
sub_set_msp_t_paths

In [ ]:
# msp_t_df = pd.DataFrame(sub_set_msp_t_paths, columns=['MSP T Paths'])

In [ ]:
x = []
    
for msp_t_path_idx in range(len(sub_set_msp_t_paths)):
    # print(msp_t_path_idx)
    msp_t_path = msp_t_paths[msp_t_path_idx]
    
    specific_frame = Image.open(msp_t_path)
    # print(np.shape(specific_frame))

    inputs = processor(images=specific_frame, return_tensors="pt")
    x.append(inputs)

In [ ]:
# x

In [ ]:
import pickle

# Input

In [ ]:
sub_set_msp_t = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' +'inputs-mspT_subset.txt'
sub_set_msp_t

In [ ]:
with open(sub_set_msp_t, 'wb') as f:
    pickle.dump(x, f)  

In [ ]:
reading_sub_set_msp_t = pd.read_pickle(sub_set_msp_t)

In [ ]:
# reading_sub_set_msp_t

# Hidden

In [ ]:
all_last_hidden_states = []

for specific_frame in range(len(reading_sub_set_msp_t)):
    inputs = reading_sub_set_msp_t[specific_frame]
    outputs = model(**inputs)
    
    # last_hidden_states == representation (1 and 2 with GradCam)
    last_hidden_states = outputs.last_hidden_state
    all_last_hidden_states.append(last_hidden_states)

In [ ]:
msp_t_hidden_states = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'hiddenStates-mspT_subset.txt'

In [ ]:
with open(msp_t_hidden_states, 'wb') as f:
    pickle.dump(all_last_hidden_states, f)  

In [ ]:
reading_msp_t_hidden_states = pd.read_pickle(msp_t_hidden_states)

In [ ]:
# reading_msp_t_hidden_states

In [ ]:
extracted_features_per_video = torch.empty((0, 197, 768))
save_efs = []
for specific_frame_hs in range(len(reading_msp_t_hidden_states)):
    
    hs = reading_msp_t_hidden_states[specific_frame_hs]
    extracted_features_per_video = torch.vstack((extracted_features_per_video, hs))
    
    save_efs.append(extracted_features_per_video)

In [ ]:
# save_efs

In [4]:
sample_msp_t_extracted_features = BASE + CLASS_PATH + DATASET_PATH + 'subsets_vit/' + 'extractedFeatures-mspT.txt'

In [ ]:
torch.save(save_efs, sample_msp_t_extracted_features)

In [5]:
fe_msp_t = torch.load(sample_msp_t_extracted_features)

In [6]:
fe_msp_t

[tensor([[[ 0.2440, -0.0993,  0.0610,  ..., -0.5175,  0.1272,  0.0336],
          [ 0.0743, -0.3381,  0.1514,  ..., -0.2165,  0.1043, -0.0217],
          [ 0.0185, -0.1522,  0.0503,  ..., -0.1910,  0.0250, -0.0989],
          ...,
          [-0.0234, -0.0225,  0.2797,  ..., -0.3946, -0.0294, -0.1056],
          [-0.1380, -0.1529,  0.2848,  ..., -0.4304, -0.1265, -0.1519],
          [-0.1926, -0.1368,  0.2996,  ..., -0.4281, -0.1503, -0.0723]]],
        requires_grad=True),
 tensor([[[ 0.2440, -0.0993,  0.0610,  ..., -0.5175,  0.1272,  0.0336],
          [ 0.0743, -0.3381,  0.1514,  ..., -0.2165,  0.1043, -0.0217],
          [ 0.0185, -0.1522,  0.0503,  ..., -0.1910,  0.0250, -0.0989],
          ...,
          [-0.0234, -0.0225,  0.2797,  ..., -0.3946, -0.0294, -0.1056],
          [-0.1380, -0.1529,  0.2848,  ..., -0.4304, -0.1265, -0.1519],
          [-0.1926, -0.1368,  0.2996,  ..., -0.4281, -0.1503, -0.0723]],
 
         [[ 0.2038, -0.0311, -0.0575,  ..., -0.6343,  0.0448, -0.0163],
